# Download Data from Planetary Computer


## Import Dependencies

In [1]:
import geoai

## Retrieve Collections

In [2]:
collections = geoai.pc_collection_list()
collections

Retrieved 126 collections from Planetary Computer


,id,title,description
42,3dep-lidar-classification,USGS 3DEP Lidar Classification,This collection is derived from the [USGS 3DEP...
28,3dep-lidar-copc,USGS 3DEP Lidar Point Cloud,This collection contains source data from the ...
3,3dep-lidar-dsm,USGS 3DEP Lidar Digital Surface Model,This collection is derived from the [USGS 3DEP...
43,3dep-lidar-dtm,USGS 3DEP Lidar Digital Terrain Model,This collection is derived from the [USGS 3DEP...
41,3dep-lidar-dtm-native,USGS 3DEP Lidar Digital Terrain Model (Native),This collection is derived from the [USGS 3DEP...
...,...,...,...
16,terraclimate,TerraClimate,[TerraClimate](http://www.climatologylab.org/t...
55,us-census,US Census,The [2020 Census](https://www.census.gov/progr...
85,usda-cdl,USDA Cropland Data Layers (CDLs),The Cropland Data Layer (CDL) is a product of ...
90,usgs-lcmap-conus-v13,USGS LCMAP CONUS Collection 1.3,"The [Land Change Monitoring, Assessment, and P..."


## Search NAIP Imagery

In [3]:
# Initialize a map
Map = geoai.Map()
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [4]:
Map.user_roi_bounds()

In [5]:
items = geoai.pc_stac_search(
    collection='naip',
    bbox=[-76.6308, 39.2798, -76.5916, 39.3023],
    time_range='2013-01-01/2014-12-31'
)
items

Found 2 items matching search criteria


[<Item id=md_m_3907644_sw_18_1_20130917_20131112>,
 <Item id=md_m_3907643_se_18_1_20130917_20131112>]

## Visualize NAIP Imagery

In [6]:
geoai.pc_item_asset_list(items[0])

['image', 'metadata', 'thumbnail', 'tilejson', 'rendered_preview']

In [7]:
geoai.view_pc_item(item=items[0])

In [8]:
url = 'https://planetarycomputer.microsoft.com/api/stac/v1/collections/naip/items/md_m_3907644_sw_18_030_20230525_20231018'
geoai.view_pc_item(url)

## Download NAIP Imagery

In [9]:
downloaded = geoai.pc_stac_download(
    items=items[0],
    output_dir='data',
    assets=['image', 'thumbnail']
)

Processing STAC item: md_m_3907644_sw_18_1_20130917_20131112
Skipping existing asset: image -> data/md_m_3907644_sw_18_1_20130917_20131112_image.tif
Skipping existing asset: thumbnail -> data/md_m_3907644_sw_18_1_20130917_20131112_thumbnail.jpg

Downloaded 2 assets for 1 items


## Search Landcover Data

In [10]:
items = geoai.pc_stac_search(
    collection='chesapeake-lc-13',
    bbox=[-76.6308, 39.2798, -76.5916, 39.3023],
    time_range='2013-01-01/2014-12-31',
    max_items=10
)

items

Found 1 items matching search criteria


[<Item id=Baywide_13Class_20132014_E1640000_N1970000>]

## Visualize Landcover Data

In [11]:
geoai.pc_item_asset_list(items[0])

['data', 'tilejson', 'rendered_preview']

In [12]:
geoai.view_pc_item(item=items[0], colormap_name='tab10', basemap='SATELLITE', backend='ipyleaflet')

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Download Land Cover Data

In [13]:
geoai.pc_item_asset_list(items[0])

['data', 'tilejson', 'rendered_preview']

In [14]:
geoai.pc_stac_download(
    items=items[0],
    output_dir='data',
    assets=['data', 'rendered_preview']
)

Processing STAC item: Baywide_13Class_20132014_E1640000_N1970000
Skipping existing asset: data -> data/Baywide_13Class_20132014_E1640000_N1970000_data.tif
Skipping existing asset: rendered_preview -> data/Baywide_13Class_20132014_E1640000_N1970000_rendered_preview.png

Downloaded 2 assets for 1 items


{'Baywide_13Class_20132014_E1640000_N1970000': {'data': 'data/Baywide_13Class_20132014_E1640000_N1970000_data.tif',
  'rendered_preview': 'data/Baywide_13Class_20132014_E1640000_N1970000_rendered_preview.png'}}

In [15]:
ds = geoai.read_pc_item_asset(items[0], asset='data', output='data/landcover.tif')
ds

Saving asset 'data' to data/landcover.tif...


Asset 'data' saved successfully.


<xarray.DataArray (band: 1, y: 10000, x: 10000)> Size: 100MB
[100000000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 80kB 1.64e+06 1.64e+06 ... 1.65e+06 1.65e+06
  * y            (y) float64 80kB 1.98e+06 1.98e+06 ... 1.97e+06 1.97e+06
    spatial_ref  int64 8B 0
Attributes:
    DataType:            Generic
    AREA_OR_POINT:       Area
    RepresentationType:  THEMATIC
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0

## Search Landsat Data

In [25]:
items = geoai.pc_stac_search(
    collection='landsat-c2-l2',
    bbox=[-76.6308, 39.2798, -76.5916, 39.3023],
    time_range='2024-04-01/2024-08-31',
    query={'eo:cloud_cover': {'lt': 1}},
    max_items=10
)

items

Found 2 items matching search criteria


[<Item id=LC08_L2SP_015033_20240825_02_T1>,
 <Item id=LC09_L2SP_015033_20240716_02_T1>]

In [28]:
geoai.pc_item_asset_list(items[0])

['qa',
 'ang',
 'red',
 'blue',
 'drad',
 'emis',
 'emsd',
 'trad',
 'urad',
 'atran',
 'cdist',
 'green',
 'nir08',
 'lwir11',
 'swir16',
 'swir22',
 'coastal',
 'mtl.txt',
 'mtl.xml',
 'mtl.json',
 'qa_pixel',
 'qa_radsat',
 'qa_aerosol',
 'tilejson',
 'rendered_preview']

In [29]:
geoai.view_pc_item(item=items[0], assets=['red', 'green', 'blue'])

In [30]:
geoai.view_pc_item(item=items[0], assets=['nir08', 'red', 'green'])

In [42]:
geoai.view_pc_item(
    item=items[0], 
    expression='(nir08-red)/(nir08+red)',
    rescale='-0.7, 0.7',
    colormap_name='rdylgn',
    name='NDVI'
)   

## Download Landsat Data

In [43]:
geoai.pc_stac_download(
    items=items[0],
    output_dir='data',
    assets=['nir08','red', 'green', 'blue'],
)

Processing STAC item: LC08_L2SP_015033_20240825_02_T1



Downloaded 4 assets for 1 items


{'LC08_L2SP_015033_20240825_02_T1': {'red': 'data/LC08_L2SP_015033_20240825_02_T1_red.tif',
  'blue': 'data/LC08_L2SP_015033_20240825_02_T1_blue.tif',
  'green': 'data/LC08_L2SP_015033_20240825_02_T1_green.tif',
  'nir08': 'data/LC08_L2SP_015033_20240825_02_T1_nir08.tif'}}